# CaReNet - CapsNet based regression network

## Application: Facial keypoint Detection using CaReNet

This notebook contains the implementations details of the CaReNet in the NCVPRIPG <a href="https://github.com/abhinay-b/Deep-Learning">publication</a>. 

We propose a capsule based regression network (CaReNet),a framework that is based on capsule networks (CapsNet), rather than on the conventional convolutional neural networks (CNNs) to determine estimates of continuous variables. The core principles of CaReNet remain that of routing-by-agreement and translation equivariance proposed inthe CapsNet architecture for classification problems. However, unlike the CapsNet architecture, the final layer capsules in CaReNet architecture capture the number of values to regress in their dimensionality. An output vector returned by each of these capsules contains all the regressed values while the corresponding activity vector, determined by squashing an output vector, captures the likelihood of the regressed values being present in the corresponding capsule. 

## Architecture:

### Encoder:
<img src="../Images/2DCaReNet.jpg" width="1200" />

The architecture of our 2D capsule-based regression network (2D CaReNet) for facial key point detection has an encoder structure with the following layers similar to that in <a href= "https://papers.nips.cc/paper/6975-dynamic-routing-between-capsules.pdf"> CapsNet</a>:

- **Convolutional layer**: This layer has 256 convolution kernels of size 9 $\times$ 9 with a stride of 1 and ReLU activation unit.
- **PrimaryCapsule layer**: This layer is made up of 32 capsules and each capsule has 8 convolutional units with each having a kernel of size 16 $ \times $ 16 and stride of 12.
- **RoutingCapsule layer**: This layer is made up of ten 30D capsules. Each of the capsules obtains the input from all the capsules in PrimaryCapsules. Further, each capsule outputs a vector that gives the corresponding activity vector of the capsule when squashed using the squash function given in equation. As the length of an activity vector determines the probability that an entity exists at the corresponding capsule, we use the squashed vectors or the activity vectors to determine the most probable capsule to be routed dynamically. The non-squashed values are not used for any other purpose in this layer.
- **DetectionCapsule layer**: This layer is also made up of ten 30D capsules and the squashed vectors are used to dynamically determine the capsule to be routed to. Further, in this layer, the \textbf{non-squashed vector with the highest length} in the corresponding activity vector is considered as the regressed output vector that captures the expected keypoints in its dimensions. Note that both the non-squashed values and the squashed values are used here. The squashed values give the probability of routing to a particular capsule (as proposed in <a href= "https://papers.nips.cc/paper/6975-dynamic-routing-between-capsules.pdf"> CapsNet</a>) while non-squashed values give the expected key points regressed by that capsule(proposed in this work).

### Decoder:
The decoder structure is very similar to the one in <a href= "https://papers.nips.cc/paper/6975-dynamic-routing-between-capsules.pdf"> CapsNet</a>:

<img src="../Images/decoder.png" width="500" />

### Import Necessary Modules

In [1]:
import torch
import torch.nn as nn
#from capsnet_utils import *
import torch.optim
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, sampler
import torchvision.utils as vutils
from pandas.io.parsers import read_csv
import numpy as np
import random
import time
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt

### Squash function:

In [2]:
def my_squash(s, dim = 2):
    
    s_norm = torch.norm(s, p = 2, dim = dim, keepdim = True)
    s_norm_sqr = s_norm ** 2     
    scalar = s_norm_sqr / (1. + s_norm_sqr)
    v = scalar * (s / s_norm)
    return v

In [3]:
def predict_activity(W, u):
    
    num_capsules_higher = W.size(2)
    
    # size of W: batch_size x # capsules in lower_layer x # capsules in higher_layer 
    #                       x capsule_dim in higher layer x capsule_dim in lower layer
    # size of u: batch_size x # capsules in lower_layer  
    #                       x capsule_dim in lower layer x 1
    # if we stack u along dim 2 # capsules in higher_layer times, size of u will become
    #           batch_size x # capsules in lower_layer x # capsules in higher_layer
    #                      x capsule_dim in lower layer x 1
    # so we can directly invoke torch.matmul to do the multiplication of two tensors and get u_hat whose size is
    #           batch_size x # capsules in lower_layer x # capsules in higher_layer 
    #                      x capsule_dim in higher layer x 1
    
    u = torch.stack([u]*num_capsules_higher, dim = 2)
    u_hat = torch.matmul(W, u) 
    return u_hat

### Dynamic routing algorithm:

In [4]:
def dynamic_routing(u_hat, r):
    """
    Ip:
        u_hat: batch_size x #caps in layer l x #caps in layer l+1 x cap_dim in layer l+1 x 1
        r: number of routing iterations    
    Return:
        v: activity vectors in layer l+1.
           size of v is batch_size x #caps in layer l+1 x cap_dim in layer l+1 x 1
    """
    batch_size = u_hat.size(0)
    b = Variable(torch.zeros(1, u_hat.size(1), u_hat.size(2), 1))
    if torch.cuda.is_available():
        b = b.cuda()
    for riter in range(r):
        c = softmax(b, dim = 1) #see eqn(3)
        c = torch.cat([c] * batch_size, dim = 0).unsqueeze(4) # to take advantage of python 
                                                                # broadcasting to do step 5 in Fig 2
        s = torch.sum(c * u_hat, dim = 1) # step 5
        v = my_squash(s, dim = 2) # step 6
        v_temp = torch.stack([v] * u_hat.size(1), dim = 1) # required for vectorizing step 7 in Fig 2
        b = b + torch.matmul(u_hat.transpose(3, 4), v_temp).squeeze(4).mean(dim = 0, keepdim = True)
                                                           # step 7
    return v,s  

### Primary Capsule layer:

In [ ]:
torch.manual_seed(23)
class PrimaryCapsuleLayer(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        # create primary capsule layer
        #primary capsule has 32 channels of (8D) capsules and each capsule has 8 convolutional units
        self.conv_units = nn.ModuleList([nn.Conv2d(256, 32, kernel_size = (16, 16),
                                                          stride = (12, 12)) for i in range(8)])
    
    def forward(self, x):
        
        # forward prop through primary capsule layer          
        outputs = [m(x) for i, m in enumerate(self.conv_units)] # a list of 8 outputs
        outputs = torch.stack(outputs, dim = 4) # stack all of them along dim 4
        #assert capsules.size() == [x.size(0), 32, 6, 6, 8]
        s = outputs.view(x.size(0), -1, 8).unsqueeze(dim = 3) 
                                            # reshape to size batch_size x 1152 x 8 x 1
        v = my_squash(s, 2) # squash along dim 2           
        #assert capsules.size() == [x.size(0), 1152, 8, 1]
        return v

### Routing Capsule layer:

In [5]:
class RoutingCapsuleLayer(nn.Module):
    
    def __init__(self, lowerCapsCount, higherCapsCount, higherCapsDim, lowerCapsDim, iterations):
        
        super().__init__()
        # digits capsule; so create W as a learnable set of parameters initialized randomly
        self.W = nn.Parameter(torch.randn(1, lowerCapsCount, higherCapsCount, higherCapsDim, lowerCapsDim)) # 1st dim is 1 since right now we dont
                                                               # know batch_size
        self.routing_iterations = iterations
    
    def forward(self, x):
        #forward prop through digits capsule by prediciting activity and routing
        u_hat = predict_activity(self.W, x)
        v, s = dynamic_routing(u_hat, self.routing_iterations)
        return v, s   
    

In [6]:
def mask(x):
    
    batch_size = x.size(0)
    #masked_x = Variable(torch.zeros(x.size()))
    #if torch.cuda.is_available():
     #   masked_x = masked_x.cuda()
    v_norm = torch.norm(x, p = 2, dim = 2)   
    _, max_index = v_norm.max(dim = 1)
    #max_index = max_index.numpy().squeeze().list()
    #masked_x[range(batch_size)] = x[range(batch_size), max_index]
    max_index = max_index.data    
    masked_v = []
    max_v_indices = []
    for batch_ix in range(batch_size):
        # Batch sample
        sample = x[batch_ix]
        # Masks out the other capsules in this sample.
        v = Variable(torch.zeros(sample.size()))
        if torch.cuda.is_available():
            v = v.cuda()
        # Get the maximum capsule index from this batch sample.
        max_caps_index = max_index[batch_ix]
        v[max_caps_index] = sample[max_caps_index]
        max_v_indices.append(max_caps_index)
        masked_v.append(v) # append v to masked_v

    # Concatenates sequence of masked capsules tensors along the batch dimension.
    masked = torch.stack(masked_v, dim=0)
    max_i = torch.stack(max_v_indices, dim = 0)
    #print(masked.size())
    return masked, max_i


In [7]:
def extract(s, indices):
    batch_size = s.size(0)
    max_s_values = []
    for batch_ix in range(batch_size):
        sample = s[batch_ix]
        max_s_values.append(sample[indices[batch_ix]])
    extracted = torch.stack(max_s_values, dim = 0)
    return extracted

### Whole Network:

In [8]:
class CapsNet(nn.Module):
    
    def __init__(self, use_reconst= True):
        
        super().__init__() 
        self.conv = nn.Conv2d(1, 256, kernel_size = (9, 9), stride = (1, 1))        
        self.relu1 = nn.ReLU()
        self.primary_capsule = PrimaryCapsuleLayer()
        self.detection_capsule1 = RoutingCapsuleLayer(1568, 10, 30, 8, 3)        
        self.detection_capsule2 = RoutingCapsuleLayer(10, 10, 30, 30, 3)  
        self.use_reconst= use_reconst
        
        # for margin loss
        self.loss_criterion = nn.MSELoss(size_average = True)
        
        if (self.use_reconst):
            self.fcReconst1= nn.Linear(30 * 10,1024)
            self.reluReconst1= nn.ReLU()
            self.fcReconst2= nn.Linear(1024,4096)
            self.reluReconst2= nn.ReLU()
            self.fcReconst3= nn.Linear(4096, 9216)
            self.sigmoid= nn.Sigmoid()
               
    def forward(self, x):
        
        x = self.conv(x)        
        x = self.relu1(x)
        x = self.primary_capsule(x)        
        x, _ = self.detection_capsule1(x) 
        #Here's our innovation; return squashed and non-sqauashed values
        x, s = self.detection_capsule2(x)
        return x,s
    
    def margin_loss(self, model_output, ground_truth):
        model_output = model_output.view(-1,30,1)
        return self.loss_criterion(model_output, ground_truth)
        
    def reconstruction_loss(self, reconstructed, original, size_average= True):
        #square
        rec_loss = (reconstructed - original.view(reconstructed.size(0), -1)) ** 2
        #sum
        rec_loss = torch.sum(rec_loss, dim = 1)
        #mean
        if (size_average):
            rec_loss= rec_loss.mean()
        return rec_loss
    
    def model_loss(self, x,s, ground_truth, input_image, size_average= True):
        
        #output_norm = torch.norm(model_output, p = 2, dim = 2) 
        masked, indices = mask(x)
        extracted = extract(s, indices)
        # calculate margin loss
        loss = self.margin_loss(extracted, ground_truth)
        # if use reconstruction, then reconstruct and add the loss
        rec_loss = None
        if (self.use_reconst):
            x = masked.view (-1,10 * 30)
            x = self.fcReconst1(x)
            x = self.reluReconst1(x)
            x = self.fcReconst2(x)
            x = self.reluReconst2(x)
            x = self.fcReconst3(x)
            x = self.sigmoid(x)
            rec_loss = self.reconstruction_loss(x,input_image, size_average)
            print(loss, rec_loss)
            loss = loss + 0.0005 * rec_loss
        
        return loss

In [9]:
def softmax(input, dim=1):
    """
    Apply softmax to specific dimensions. Not released on PyTorch stable yet
    as of November 6th 2017
    https://github.com/pytorch/pytorch/issues/3235
    """
    transposed_input = input.transpose(dim, len(input.size()) - 1)
    softmaxed_output = F.softmax(
        transposed_input.contiguous().view(-1, transposed_input.size(-1)))
    return softmaxed_output.view(*transposed_input.size()).transpose(dim, len(input.size()) - 1)

In [10]:
cn_model= CapsNet(use_reconst= True)
if (torch.cuda.is_available()):
    cn_model.cuda()

In [11]:
from functools import wraps
def decorator(f):
    @wraps(f)
    def wrapper(self, i):
        sample = f(self, i)
        if self.conv:
            sample['image'] = sample['image'].reshape(1, 96, 96)
            if self.transform:
                sample = self.transform(sample)  
        sample['image'] = torch.from_numpy(sample['image'].copy())
        if not self.test:
            sample['keypoints'] = torch.from_numpy(sample['keypoints'])
        return sample
    return wrapper

def train_valid_split(dataset, test_size = 0.2, shuffle = False, random_seed = 0):
    """ Return a list of splitted indices from a DataSet.
    Indices can be used with DataLoader to build a train and validation set.
    
    Arguments:
        A Dataset
        A test_size, as a float between 0 and 1 (percentage split) or as an int (fixed number split)
        Shuffling True or False
        Random seed
    """
    length = len(dataset)
    indices = list(range(1,length))
    
    if shuffle == True:
        random.seed(random_seed)
        random.shuffle(indices)
    
    if type(test_size) is float:
        split = int(test_size * length)
    elif type(test_size) is int:
        split = test_size
    else:
        raise ValueError('%s should be an int or a float' % str)
        
    return indices[split:], indices[:split]

def load(csvfile, test_split = 0.2, test = False, conv = False, transform = None, cols = None):
    
    X = FacialKeypointDataset(csvfile, test=test, conv=conv, transform=transform, cols=cols)
    
    if not test:
        # Creating a validation split
        if not transform:
            train_idx, valid_idx = train_valid_split(X, test_split, shuffle = True)
            train_sampler = sampler.SubsetRandomSampler(train_idx)
            valid_sampler = sampler.SubsetRandomSampler(valid_idx)

            #print('Train size: ', len(train_sampler), '\tValidation Size: ', len(valid_sampler))
        
            # Both dataloader loads from the same dataset but with different indices
            train_loader = DataLoader(X,
                      batch_size=16,
                      sampler=train_sampler,
                      num_workers=8)

            valid_loader = DataLoader(X,
                      batch_size=16,
                      sampler=valid_sampler,
                      num_workers=8)

            dataloaders = {'train':train_loader, 'valid':valid_loader}
            dataset_sizes = {'train':len(train_sampler), 'valid':len(valid_sampler)}
        
        else:
            X_valid = FacialKeypointDataset(csvfile, test, conv, transform = None, cols = cols)
            train_loader = DataLoader(X,
                      batch_size=16,
                      num_workers=8)

            valid_loader = DataLoader(X_valid,
                      batch_size=16,
                      num_workers=8)
            
            dataloaders = {'train':train_loader, 'valid':valid_loader}
            dataset_sizes = {'train':len(X), 'valid':len(X_valid)}
    else:
        batch_size = 16
        dataloaders = DataLoader(X, batch_size = batch_size, drop_last = True)
        dataset_sizes = {'test' : len(dataloaders) * batch_size}
    
    return dataloaders, dataset_sizes

class FacialKeypointDataset(Dataset):
    
    def __init__(self, csv_file, test = False, conv = False, transform = None, cols = None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.            
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.face_keypoint_frame = read_csv(csv_file)
        self.cols = cols
        
        if self.cols:  # get a subset of columns
            self.face_keypoint_frame = self.face_keypoint_frame[list(self.cols) + ['Image']]
        
        self.face_keypoint_frame = self.face_keypoint_frame.dropna() #drop rows with missing entries
        self.face_keypoint_frame['Image'] = self.face_keypoint_frame['Image'] \
                        .apply(lambda im: np.fromstring(im, sep = ' ', dtype = np.float32)) 
        
        self.test = test
        self.conv = conv
        self.transform = transform
                
    def __len__(self):
        
        return len(self.face_keypoint_frame)
    
   
    @decorator
    def __getitem__(self, i):
        
        image = self.face_keypoint_frame.iloc[i, -1] / 255

        keypoints = self.face_keypoint_frame.iloc[i, :-1].as_matrix().astype(np.float32)
        keypoints = (keypoints - 48) / 48 #normalize between [-1, 1]; given range is [0 95]
        sample = {'image': image, 'keypoints': keypoints}
            
        return sample

In [12]:
# Run this cell to load the data
dataloaders, dataset_sizes = load('../Dataset/training.csv', test_split = 0.2, conv = True)

In [13]:
def train_model(model, data_loader, optimizer, save_file, scheduler = None):
    
    since = time.time()
    train_loss_history = []
    valid_loss_history = []
    num_epochs = model.num_epochs
    num_batches = len(data_loader)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for is_train in [True, False]:
            model.train(is_train)
            if scheduler:
                scheduler.step()
            running_loss = 0.0

            if (is_train):
                phase = "train"
            else:
                phase = "valid"
            for data in data_loader[phase]:
                inputs = data['image']
                labels = data['keypoints']
                #inputs, labels = Variable(inputs), Variable(labels)
                #do not uncomment the four lines below; we will be working with CPU

                if torch.cuda.is_available():
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)
                #print(inputs.size())
                optimizer.zero_grad()
                outputs, s = model(inputs)
                
                loss = model.model_loss(outputs,s,labels,inputs)
                if (is_train):
                    loss.backward()
                    optimizer.step()

                running_loss += loss.data[0]
             
            epoch_loss = running_loss / num_batches
            if (is_train):
                train_loss_history.append(epoch_loss)
            else:
                valid_loss_history.append(epoch_loss)
            print('{} Loss: {:.8f}'.format(
                    phase, epoch_loss))
            if (not is_train):
                print('Train Loss / Valid Loss: {:.6f}'.format(
                    train_loss_history[-1] / valid_loss_history[-1]))
        
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
            time_elapsed // 60, time_elapsed % 60))
    torch.save(model.state_dict(), save_file)
    torch.save(train_loss_history, save_file + '_loss_history')
    return

In [14]:
cn_model.num_epochs = 1
optimizer= torch.optim.Adam(cn_model.parameters(), lr=0.0001)
train_model(cn_model, dataloaders, optimizer, 'trainedModelCF5' + str(cn_model.num_epochs) + '.pth')

Epoch 0/0
----------
Variable containing:
 0.1571
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 497.7518
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1541
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 460.0690
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1396
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 535.0803
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1218
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 523.2101
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1285
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 533.4300
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1267
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containing:
 497.5692
[torch.cuda.FloatTensor of size 1 (GPU 0)]

Variable containing:
 0.1211
[torch.cuda.FloatTensor of size 1 (GPU 0)]
 Variable containin

Process Process-4:
Process Process-2:
Process Process-6:
Process Process-7:
Process Process-8:
Process Process-1:
Process Process-5:
Traceback (most recent call last):
Process Process-3:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/local/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    s

KeyboardInterrupt: 

In [16]:
def show_keypoints(image, regressed_keypoints, gt_keypoints):
    print(gt_keypoints)
    """Show image with keypoints"""    
    regressed_keypoints = regressed_keypoints * 48 + 48
    gt_keypoints = gt_keypoints * 48 + 48
    print (regressed_keypoints)
    plt.imshow(image, cmap = 'gray')
    plt.scatter(regressed_keypoints[0::2], regressed_keypoints[1::2], marker='.', c='r')
    plt.scatter(gt_keypoints[0::2], gt_keypoints[1::2], marker='.', c='g')

def predict_plot_conv(model, test_file, model_file, cols = None, num_output_units = 30):
                                                    #predict and plot for one batch
    test_loader, test_size = load(test_file, test = True, conv = True, cols = cols)
    #model.load_state_dict(torch.load(model_file, map_location=lambda storage, loc: storage))
    #model.eval()
    
    batch_size = int(test_size['test'] / len(test_loader))
    y_pred = torch.zeros(len(test_loader), batch_size, num_output_units)
    
    for j, test_data in enumerate(test_loader):
        v,s = model(Variable(test_data['image'].cuda()))
        _, indexOfMaxV = mask(v)
        extracted = extract(s,indexOfMaxV)
        y_pred[j] = extracted.view(-1,30,1).data
        #y_pred[j] =  torch.norm(model(Variable(test_data['image'].cuda())).data,p=2,dim=2)
        
        if j == 0:
            fig = plt.figure(figsize=(12, 12))
            fig.subplots_adjust(
                left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

            for i in range(batch_size):
                ax = fig.add_subplot(int(np.sqrt(batch_size)) + 1, int(np.sqrt(batch_size)) + 1, 
                                 i + 1, xticks=[], yticks=[])
                show_keypoints(test_data['image'][i].numpy().reshape(96, 96), y_pred[j, i, :].numpy(),
                               test_data['keypoints'][i].numpy())

In [ ]:
%matplotlib inline
#testmodel= CapsNet()
#if (torch.cuda.is_available()):
    #model.cuda()
    
cn_model.use_reconst = False
predict_plot_conv(cn_model,'../Dataset/training.csv','trainedModelCF510.pth')
#test_fc_model()